Install

In [1]:
!pip install -q rank_bm25
!pip -q install pyvi
!pip -q install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.4 MB/s eta 0:00:00


In [2]:
import re
import string
from string import punctuation

from pyvi import ViTokenizer, ViPosTagger
import Levenshtein

import ast
from rank_bm25 import BM25Okapi

import pandas as pd
import numpy as np

Testing data

In [4]:
test_data = pd.read_csv("/content/public_test.csv")
test_data

,id,question,option_1,option_2,option_3,option_4,option_5,option_6
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN
...,...,...,...,...,...,...,...,...
95,level4_4,Tiểu Đường là nữ ca sĩ nổi tiếng ở Trung Quốc....,Viêm họng do viêm amidan,Tiểu đường,Ung thư vòm họng,Trĩ nội,NaN,NaN
96,level4_9,Mai hiện mang thai tháng thứ 9. Có rất nhiều b...,Herpes sinh dục,Lậu,Chlamydia,Giang mai,NaN,NaN
97,level4_27,Ông Biền năm nay 73 tuổi. Trong một bữa cơm gi...,U não,Thiếu mãu não,Tắc mạch máu não,Tai biến mạch máu não,NaN,NaN
98,level4_28,U não là tình trạng các khối u hình thành tron...,Đều là các bệnh nguy hiểm,Đều là ung thư,Nguyên nhân chính xác không thể xác định,Xảy ra nhiều nhất ở người già,NaN,NaN


In [5]:
opts = []
for i, j in test_data.iterrows():
  opts.append([re.sub(r"[A-E]\.", "", val).strip().lower() for val in test_data.iloc[i, 2:].values if type(val) == str])

opt_list = []
for opt in opts:
  l = []
  for o in opt:
    l += o.split()
  opt_list.append(l)

test_data['opts'] = opt_list
test_data

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,opts
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN,"[tuần, 10, tuần, 20, tuần, 30, tuần, 40]"
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN,"[5, tuần, 15, tuần, 25, tuần, 35, tuần]"
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN,"[2, 3, 4, 5]"
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN,"[có, không]"
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN,"[nhiễm, trùng, đường, tiết, niệu, ung, thư, ti..."
...,...,...,...,...,...,...,...,...,...
95,level4_4,Tiểu Đường là nữ ca sĩ nổi tiếng ở Trung Quốc....,Viêm họng do viêm amidan,Tiểu đường,Ung thư vòm họng,Trĩ nội,NaN,NaN,"[viêm, họng, do, viêm, amidan, tiểu, đường, un..."
96,level4_9,Mai hiện mang thai tháng thứ 9. Có rất nhiều b...,Herpes sinh dục,Lậu,Chlamydia,Giang mai,NaN,NaN,"[herpes, sinh, dục, lậu, chlamydia, giang, mai]"
97,level4_27,Ông Biền năm nay 73 tuổi. Trong một bữa cơm gi...,U não,Thiếu mãu não,Tắc mạch máu não,Tai biến mạch máu não,NaN,NaN,"[u, não, thiếu, mãu, não, tắc, mạch, máu, não,..."
98,level4_28,U não là tình trạng các khối u hình thành tron...,Đều là các bệnh nguy hiểm,Đều là ung thư,Nguyên nhân chính xác không thể xác định,Xảy ra nhiều nhất ở người già,NaN,NaN,"[đều, là, các, bệnh, nguy, hiểm, đều, là, ung,..."


Corpus

In [20]:
!gdown -q 1Wx_KP0CXbBR_SjdkunTswUskkzZkujPh

In [21]:
import pandas as pd
corpus = pd.read_csv('final_answer_dict_v3.csv').iloc[:,1:]
corpus

,ques,ans
0,Định nghĩa Đau khớp,"Định nghĩa Đau khớp Đau khớp, là cảm giác khó ..."
1,Đau khớp. Vị trí đau khớp,Đau khớp. Vị trí đau khớp Khớp gối Những cơn đ...
2,Đau khớp. Nguyên nhân gây đau nhức khớp,Đau khớp. Nguyên nhân gây đau nhức khớp Nguyên...
3,Đau khớp. Chẩn đoán đau khớp,Đau khớp. Chẩn đoán đau khớp Bác sĩ khám sức k...
4,Đau khớp. Đối tượng dễ mắc bệnh đau khớp,Đau khớp. Đối tượng dễ mắc bệnh đau khớp Thoái...
...,...,...
9642,Tắc ống dẫn tinh. 5. Tắc ống phóng tinh,Tắc ống dẫn tinh. Tắc ống phóng tinh Điều trị ...
9643,Mỡ máu cao. Ezetimibe,Mỡ máu cao. Ezetimibe Ezetimibe là một loại th...
9644,Đau khớp khuỷu tay. Vật lý trị liệu,Đau khớp khuỷu tay. Vật lý trị liệu Vật lý trị...
9645,Bướu giáp lan tỏa. Đối với người bị bướu giáp ...,Bướu giáp lan tỏa. Đối với người bị bướu giáp ...


In [22]:
def preprocessing_text(text):
  for punc in punctuation:
    text = text.replace(punc, ' ')
  for p in ['“', '”', '…', '–']:
    text = text.replace(p, ' ')
  text = re.sub(' +', ' ', text).strip()
  return text.strip().lower()
context_list = [preprocessing_text(key) for key in corpus['ans']]
corpus['context'] = context_list
corpus

,ques,ans,context
0,Định nghĩa Đau khớp,"Định nghĩa Đau khớp Đau khớp, là cảm giác khó ...",định nghĩa đau khớp đau khớp là cảm giác khó c...
1,Đau khớp. Vị trí đau khớp,Đau khớp. Vị trí đau khớp Khớp gối Những cơn đ...,đau khớp vị trí đau khớp khớp gối những cơn đa...
2,Đau khớp. Nguyên nhân gây đau nhức khớp,Đau khớp. Nguyên nhân gây đau nhức khớp Nguyên...,đau khớp nguyên nhân gây đau nhức khớp nguyên ...
3,Đau khớp. Chẩn đoán đau khớp,Đau khớp. Chẩn đoán đau khớp Bác sĩ khám sức k...,đau khớp chẩn đoán đau khớp bác sĩ khám sức kh...
4,Đau khớp. Đối tượng dễ mắc bệnh đau khớp,Đau khớp. Đối tượng dễ mắc bệnh đau khớp Thoái...,đau khớp đối tượng dễ mắc bệnh đau khớp thoái ...
...,...,...,...
9642,Tắc ống dẫn tinh. 5. Tắc ống phóng tinh,Tắc ống dẫn tinh. Tắc ống phóng tinh Điều trị ...,tắc ống dẫn tinh tắc ống phóng tinh điều trị t...
9643,Mỡ máu cao. Ezetimibe,Mỡ máu cao. Ezetimibe Ezetimibe là một loại th...,mỡ máu cao ezetimibe ezetimibe là một loại thu...
9644,Đau khớp khuỷu tay. Vật lý trị liệu,Đau khớp khuỷu tay. Vật lý trị liệu Vật lý trị...,đau khớp khuỷu tay vật lý trị liệu vật lý trị ...
9645,Bướu giáp lan tỏa. Đối với người bị bướu giáp ...,Bướu giáp lan tỏa. Đối với người bị bướu giáp ...,bướu giáp lan tỏa đối với người bị bướu giáp l...


Disease names

In [13]:
filename = 'disease_names.txt'
keys = pd.read_csv(filename, sep=',',header=None).iloc[0].tolist()[:-1]
disease_names = []
for k in keys:
  disease_names += k.split()
disease_names_list = list(set(disease_names))
disease_names_list[:5]

['bệnh', 'cương', 'hô', 'bị', 'mộng']

A - Adjective
C - Coordinating conjunction
E - Preposition
I - Interjection
L - Determiner
M - Numeral
N - Common noun
Nc - Noun Classifier
Ny - Noun abbreviation
Np - Proper noun
Nu - Unit noun
P - Pronoun
R - Adverb
S - Subordinating conjunction
T - Auxiliary, modal words
V - Verb
X - Unknown
F - Filtered out (punctuation)

In [14]:
def split_pharse(text):
  return [t.strip() for t in text.split(' và ')]

def split_ques(text):
  result = []
  sent = text.split('. ')[::-1]
  if len(sent) > 1:
    for s in sent:
      result += split_pharse(s)
  else:
    result += sent
  return result

def q_token(text):
  token = []
  doc = ViTokenizer.tokenize(text)
  token_doc = [str(a) for a, b in zip(*ViPosTagger.postagging(doc)) if b not in ['T', 'F', 'P', 'L', 'R', 'C', 'E', 'X', 'Nu']]
  for tk in token_doc:
    token += tk.split('_')
  return token

def ques_token(text):
  token = []
  for idx, q in enumerate(split_ques(text)):
    if idx == 0:
      token += q_token(q)
    else:
      bos = []
      for val in q_token(q):
        if any([key for key in disease_names_list if Levenshtein.ratio(val, key) >= 0.95]):
          bos+=[val]
      token += list(set(bos))
  return [t.lower() for t in token]

def bm25_searcher(question_index, corpus, bm25, indexes, top_k=1):
    question = test_data.iloc[question_index]['question']
    print('\nquestion >>>')
    print(question)

    opts = test_data.iloc[question_index]['opts']
    print('\noptions >>>')
    print(opts)

    tokenized_query = ques_token(question) + opts
    print('\ntokenized_query >>>')
    print(tokenized_query)

    top_indexs = bm25.get_top_n(tokenized_query, indexes, n=top_k)
    top_docs = []
    for top_index in top_indexs:
      top_docs.append(corpus['ans'].iloc[top_index])
    return top_docs

In [29]:
#Take a look
words = [key.split() for key in context_list]
bm25 = BM25Okapi(words)
indexs = list(range(len(words)))

question_index = 12

top_docs = bm25_searcher(question_index, corpus, bm25, indexs, top_k=5)
print('\ntop_doc >>>')
for idx, doc in enumerate(top_docs):
  print(f'\nTop {idx} >>> \n{doc} \nlen: {len(doc.split())}')


question >>>
Quỳnh là một người mẫu nổi tiếng ở Việt Nam. Với nghề người mẫu việc chăm sóc tốt da là điều cực kỳ quan trọng. Đâu là giải pháp giúp Quỳnh chăm sóc da từ bên trong?

options >>>
['ngủ', 'đủ', '4', 'tiếng/ngày', 'ăn', 'nhiều', 'đồ', 'ngọt', 'uống', 'đủ', 'nước', 'mỗi', 'ngày', '1,5', '–', '2', 'lít']

tokenized_query >>>
['là', 'giải', 'pháp', 'giúp', 'quỳnh', 'chăm', 'sóc', 'da', 'bên', 'trong', 'kỳ', 'cực', 'da', 'một', 'tiếng', 'ngủ', 'đủ', '4', 'tiếng/ngày', 'ăn', 'nhiều', 'đồ', 'ngọt', 'uống', 'đủ', 'nước', 'mỗi', 'ngày', '1,5', '–', '2', 'lít']

top_doc >>>

Top 0 >>> 
Mụn nội tiết. Chăm sóc bảo vệ da khỏi mụn nội tiết Mụn nội tiết, gây mất thẩm mỹ, có thể kéo dài dai dẳng và tái đi tái lại nhiều lần. Điều trị và chăm sóc da đúng cách sẽ hạn chế mụn Chăm sóc da từ bên ngoài Rửa mặt không quá 2 lần/ngày và sau khi đổ mồ hôi. Sử dụng sữa rửa mặt dịu nhẹ và nước ấm, không dùng nước nóng. Tránh chà xát, nặn mụn vì gây viêm hoặc làm tình trạng nặng hơn. Sử dụng mỹ phẩm đ

In [30]:
retrieved_context = []
for index, col in test_data.iterrows():
  top_docs = bm25_searcher(index, corpus, bm25, indexs, top_k=5)
  retrieved_context.append(". ".join(top_docs))
test_data['retrieved_context'] = retrieved_context


question >>>
Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?

options >>>
['tuần', '10', 'tuần', '20', 'tuần', '30', 'tuần', '40']

tokenized_query >>>
['hương', 'kiểm', 'tra', 'phát', 'hiện', 'bệnh', 'tuần', 'thứ', 'thai', 'kỳ', 'thai', 'mang', 'rau', 'lo', 'phải', 'tiền', 'đạo', 'tuần', '10', 'tuần', '20', 'tuần', '30', 'tuần', '40']

question >>>
Hương đang mang thai tuần thứ 5 và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này sau bao nhiêu tuần nữa?

options >>>
['5', 'tuần', '15', 'tuần', '25', 'tuần', '35', 'tuần']

tokenized_query >>>
['hương', 'kiểm', 'tra', 'phát', 'hiện', 'bệnh', 'tuần', 'thai', 'mang', 'rau', 'lo', 'phải', 'tiền', 'đạo', '5', 'tuần', '15', 'tuần', '25', 'tuần', '35', 'tuần']

question >>>
Có bao nhiêu loại rau tiền đạo biết rằng trong bóng đá thường mỗi đội có tối đa 3 tiền đạo trên sân?

options >>>
['2', '3', '4', '5']

tokenized_q

In [31]:
test_data

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,opts,retrieved_context
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN,"[tuần, 10, tuần, 20, tuần, 30, tuần, 40]",Rau tiền đạo. Bao nhiêu tuần thì biết được rau...
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN,"[5, tuần, 15, tuần, 25, tuần, 35, tuần]",Rau tiền đạo. Bao nhiêu tuần thì biết được rau...
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN,"[2, 3, 4, 5]",Rau tiền đạo. Dấu hiệu rau tiền đạo thường gặp...
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN,"[có, không]",Viêm gan C. Dấu hiệu nhận biết bệnh viêm gan C...
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN,"[nhiễm, trùng, đường, tiết, niệu, ung, thư, ti...","Định nghĩa Sa tinh hoàn Sa tinh hoàn, (xệ tinh..."
...,...,...,...,...,...,...,...,...,...,...
95,level4_4,Tiểu Đường là nữ ca sĩ nổi tiếng ở Trung Quốc....,Viêm họng do viêm amidan,Tiểu đường,Ung thư vòm họng,Trĩ nội,NaN,NaN,"[viêm, họng, do, viêm, amidan, tiểu, đường, un...","Đau họng. Cách trị đau họng, Tùy vào nguyên nh..."
96,level4_9,Mai hiện mang thai tháng thứ 9. Có rất nhiều b...,Herpes sinh dục,Lậu,Chlamydia,Giang mai,NaN,NaN,"[herpes, sinh, dục, lậu, chlamydia, giang, mai]",Bệnh giang mai. Biến chứng nguy hiểm của giang...
97,level4_27,Ông Biền năm nay 73 tuổi. Trong một bữa cơm gi...,U não,Thiếu mãu não,Tắc mạch máu não,Tai biến mạch máu não,NaN,NaN,"[u, não, thiếu, mãu, não, tắc, mạch, máu, não,...",Tắc mạch máu não. Triệu chứng tắc nghẽn mạch m...
98,level4_28,U não là tình trạng các khối u hình thành tron...,Đều là các bệnh nguy hiểm,Đều là ung thư,Nguyên nhân chính xác không thể xác định,Xảy ra nhiều nhất ở người già,NaN,NaN,"[đều, là, các, bệnh, nguy, hiểm, đều, là, ung,...",U não. U não ác tính (ung thư) Định nghĩa: U n...


In [32]:
test_data.to_csv("updated_test_data_v2.csv")

# Translate to English

In [33]:
import pandas as pd
test_data = pd.read_csv("updated_test_data_v2.csv")
test_data = test_data.iloc[:, 1:]
test_data

,id,question,option_1,option_2,option_3,option_4,option_5,option_6,opts,retrieved_context
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN,"['tuần', '10', 'tuần', '20', 'tuần', '30', 'tu...",Rau tiền đạo. Bao nhiêu tuần thì biết được rau...
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN,"['5', 'tuần', '15', 'tuần', '25', 'tuần', '35'...",Rau tiền đạo. Bao nhiêu tuần thì biết được rau...
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN,"['2', '3', '4', '5']",Rau tiền đạo. Dấu hiệu rau tiền đạo thường gặp...
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN,"['có', 'không']",Viêm gan C. Dấu hiệu nhận biết bệnh viêm gan C...
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN,"['nhiễm', 'trùng', 'đường', 'tiết', 'niệu', 'u...","Định nghĩa Sa tinh hoàn Sa tinh hoàn, (xệ tinh..."
...,...,...,...,...,...,...,...,...,...,...
95,level4_4,Tiểu Đường là nữ ca sĩ nổi tiếng ở Trung Quốc....,Viêm họng do viêm amidan,Tiểu đường,Ung thư vòm họng,Trĩ nội,NaN,NaN,"['viêm', 'họng', 'do', 'viêm', 'amidan', 'tiểu...","Đau họng. Cách trị đau họng, Tùy vào nguyên nh..."
96,level4_9,Mai hiện mang thai tháng thứ 9. Có rất nhiều b...,Herpes sinh dục,Lậu,Chlamydia,Giang mai,NaN,NaN,"['herpes', 'sinh', 'dục', 'lậu', 'chlamydia', ...",Bệnh giang mai. Biến chứng nguy hiểm của giang...
97,level4_27,Ông Biền năm nay 73 tuổi. Trong một bữa cơm gi...,U não,Thiếu mãu não,Tắc mạch máu não,Tai biến mạch máu não,NaN,NaN,"['u', 'não', 'thiếu', 'mãu', 'não', 'tắc', 'mạ...",Tắc mạch máu não. Triệu chứng tắc nghẽn mạch m...
98,level4_28,U não là tình trạng các khối u hình thành tron...,Đều là các bệnh nguy hiểm,Đều là ung thư,Nguyên nhân chính xác không thể xác định,Xảy ra nhiều nhất ở người già,NaN,NaN,"['đều', 'là', 'các', 'bệnh', 'nguy', 'hiểm', '...",U não. U não ác tính (ung thư) Định nghĩa: U n...


In [34]:
!pip -q install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [35]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='vi', target='en')

In [49]:
def translate_text(text):
    if isinstance(text, str):
      try:
        translated_text = translator.translate(text)
        return translated_text
      except:
        chunks = [text[x:x+250] for x in range(0, len(text), 250) if len(text[x:x+250]) > 1]
        translated_text = ' '.join(translator.translate_batch(chunks))
        return translated_text
    else:
        return ''

In [ ]:
from tqdm import tqdm

en_test_data = []
for idx, row in tqdm(test_data.iterrows()):
    id = row['id']
    question = translate_text(row['question'])
    option_1 = translate_text(row['option_1'])
    option_2 = translate_text(row['option_2'])
    option_3 = translate_text(row['option_3'])
    option_4 = translate_text(row['option_4'])
    option_5 = translate_text(row['option_5'])
    option_6 = translate_text(row['option_6'])
    context = translate_text(row['retrieved_context'])
    data = {
        'id': id,
        'question': question,
        'option_1': option_1,
        'option_2': option_2,
        'option_3': option_3,
        'option_4': option_4,
        'option_5': option_5,
        'option_6': option_6,
        'retrieved_context': context,
    }
    en_test_data.append(data)

In [54]:
en_test_df = pd.DataFrame(en_test_data)
en_test_df.to_csv("updated_test_data_v2.csv")